In [2]:
import collections

import datasets
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import tqdm
print("torchtext version : " , torchtext.__version__)
print("torch version : ",torch.__version__)
torchtext._TORCHTEXT_DEPRECATION_MSG =False

torchtext version :  0.18.0+cpu
torch version :  2.3.0+cpu


In [3]:
train_data, test_data = datasets.load_dataset("imdb", split=["train", "test"])

In [4]:
train_data,test_data

(Dataset({
     features: ['text', 'label'],
     num_rows: 25000
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 25000
 }))

In [5]:
train_data.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [6]:
train_data[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [7]:
#pip install torchtext==0.4 resolve an issue when trying to import the module
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer("basic_english")


TypeError: expected string or bytes-like object, got 'bool'

In [10]:
#example of use of the tokenizer :

tokenizer("Hello world ! How are you doing today")

NameError: name 'tokenizer' is not defined

In [11]:
def tokenize_example(example,tokenizer, max_length):
    tokens = tokenizer(example["text"])[:max_length]
    return {"tokens" : tokens}

In [12]:
max_length = 256

train_data = train_data.map(
    tokenize_example, fn_kwargs={"tokenizer" : tokenizer, "max_length" : max_length}
)

test_data = test_data.map(
    tokenize_example, fn_kwargs={"tokenizer" : tokenizer, "max_length" : max_length}
)

NameError: name 'tokenizer' is not defined

In [13]:
train_data

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [14]:
train_data.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [15]:
train_data[0]["tokens"][:25]

KeyError: 'tokens'

In [16]:
#We are going to create a validation test set which is a fraction of the real
#test set the validation one is the set we will use to test and the test set is the one  we will use in the final test 
#We create two sets because we dont want to overfit the model
test_size = 0.25

train_valid_data = train_data.train_test_split(test_size=test_size)
train_data = train_valid_data["train"]
valid_data = train_valid_data["test"]

In [17]:
#We can see the split : 
len(train_data),len(valid_data), len(test_data)


(18750, 6250, 25000)

In [31]:
from torchtext.vocab import build_vocab_from_iterator
min_freq = 5
special_token = ["<unk>","<pad>"]

vocab = build_vocab_from_iterator(
    train_data["tokens"],
    min_freq = min_freq,
    specials=special_token
)



TypeError: expected string or bytes-like object, got 'bool'

In [19]:
len(vocab)

NameError: name 'vocab' is not defined

In [20]:
vocab.get_itos()[:10]

NameError: name 'vocab' is not defined

In [21]:
unk_index = vocab["<unk>"]
pad_index = vocab["<pad>"]

print(unk_index)
print(pad_index)

NameError: name 'vocab' is not defined

In [22]:
#we can chek if a token is in our vacabulary using the in operator

"some_token" in vocab

NameError: name 'vocab' is not defined

In [23]:
vocab.set_default_index(unk_index)

NameError: name 'vocab' is not defined

In [24]:
def numericalize_example(example, vocab):
    ids = vocab.lookup_indices(example["tokens"])
    return {"ids" : ids}

In [25]:
train_data = train_data.map(numericalize_example, fn_kwargs={"vocab" : vocab})
valid_data = valid_data.map(numericalize_example,fn_kwargs={"vocab": vocab})
test_data = test_data.map(numericalize_example, fn_kwargs={"vocab": vocab})

NameError: name 'vocab' is not defined

In [26]:
train_data['text']

['To be honest, I didn\'t like "Executive Decision" - which was obvious the template - very much, but compared to this piece of crap, it looks like a masterpiece of art.<br /><br />Not only that the people moving in the film (the term actors would be an insult for all other actors) should attend more acting classes, the guy who build the setting hasn\'t even seen an aircraft from afar. It is so ridiculous, that on a 747 only 3 flight attendance are aboard, but this is only the tip of the iceberg. The film is full of illogicalness (e.g.: use sleeping-gas, then don\'t use it, then use it anyway), which dropped my rating finally to the bottom.<br /><br />You can have more fun in rearranging you sock drawer or in drilling a hole in your knee and fill it with milk.',
 'Well, "Cube" (1997), Vincenzo\'s first movie, was one of the most interesting and tricky ideas that I\'ve ever seen when talking about movies. They had just one scenery, a bunch of actors and a plot. So, what made it so speci

In [27]:
vocab.lookup_indices(train_data[0]["tokens"][:10])


NameError: name 'vocab' is not defined

In [28]:
train_data[0]['ids'][:10]

KeyError: 'ids'

In [29]:
#We then need to turn the data into matrices
train_data = train_data.with_format(type="torch",columns=["ids","label"])
valid_data = valid_data.with_format(type="torch",columns=["ids","label"])
test_data = test_data.with_format(type="torch",columns=["ids","label"])


ValueError: Columns ['ids'] not in the dataset. Current columns in the dataset: ['text', 'label']

In [30]:
train_data[0].keys()

dict_keys(['text', 'label'])